## Milestone 02

- Class: BDA700NAA - Health Analytics
- Subject: Health Analytics Project - Milestone 02
- Members: Ran Arino, Zubeka Dane Dang

### Data Import



In [1]:
# Import the kaggle library
from kaggle.api.kaggle_api_extended import KaggleApi

# Initialize the API
api = KaggleApi()
api.authenticate()

# Download the dataset
# Format: api.dataset_download_files('username/dataset-name', path='./data')
api.dataset_download_files('awsaf49/brats20-dataset-training-validation', path='./data')

print("Dataset downloaded to ./data directory")

Dataset URL: https://www.kaggle.com/datasets/awsaf49/brats20-dataset-training-validation
Dataset downloaded to ./data directory


### Data Preprocessing
